# Lesson 8: Multiple Linear Regression (using sklearn)

## Import libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

from sklearn.linear_model import LinearRegression

## Load data 

In [4]:
data = pd.read_csv("1.02.Multiple_linear_regression.csv")

In [5]:
data.head()

,SAT,GPA,"Rand 1,2,3"
0,1714,2.40,1
1,1664,2.52,3
2,1760,2.54,3
3,1685,2.74,3
4,1693,2.83,2


In [6]:
data.describe()

,SAT,GPA,"Rand 1,2,3"
count,84.000000,84.000000,84.000000
mean,1845.273810,3.330238,2.059524
std,104.530661,0.271617,0.855192
min,1634.000000,2.400000,1.000000
25%,1772.000000,3.190000,1.000000
50%,1846.000000,3.380000,2.000000
75%,1934.000000,3.502500,3.000000
max,2050.000000,3.810000,3.000000


## Create the multiple linear regression 

### Declare the independent and dependent variables 

In [8]:
x = data[["SAT","Rand 1,2,3"]]
y = data["GPA"]

### Create regression 

In [9]:
reg = LinearRegression()
reg.fit(x,y)

LinearRegression()

In [10]:
reg.coef_

array([ 0.00165354, -0.00826982])

In [11]:
reg.intercept_

0.29603261264909486

### Calculating R-squared

In [12]:
reg.score(x,y)

0.40668119528142843

Note that in sklearn we do not have a formula to calculate adjusted R-squared, which is more appropriate for 
multiple linear regression. We need to create our own formula to do this.

### Formula for adjusted R-squared 

We can use Latex form to create formulas.

$R^2_{adj.} = 1-(1-R^2)*\frac{n-1}{n-p-1}$   

$n$ - number of observations (rows), $n=84$;
$p$ - number of predictors (independent variables), $p=2$;


In [17]:
x.shape

(84, 2)

In [21]:
r2 = reg.score(x,y)
n = x.shape[0]
p = x.shape[1]

adjusted_r2 = 1-(1-r2)*(n-1)/(n-p-1)
adjusted_r2

0.39203134825134023

Because adjusted R-squared is smaller than R-squared, one of the independent variables is not needed as it does not 
have predictive power. We need to find a way to remove the dependence on this variable.

In sklearn there is a function (f_regression) that makes regression for both features (independent variables) separately.

### Feature selection 

In [23]:
from sklearn.feature_selection import f_regression

In [24]:
f_regression(x,y)

(array([56.04804786,  0.17558437]), array([7.19951844e-11, 6.76291372e-01]))

Above the first array contains the F-statistics for both independent variables and the second array contains the p-values. 

In [26]:
f_stat = f_regression(x,y)[0]
p_values = f_regression(x,y)[1]

p_values.round(3)

array([0.   , 0.676])

Having this we see the following: the fist value in the array is the p value for "SAT", 
and since p<0.5 "SAT" is relevant and important variable. p>0.5 for the "Rand 1,2,3" variable and therefore 
this variable is insignificant. 

Note that f_regression does not include interconnection between the two independent variables so 
it may not capture a full picture. It is good for simplistic cases. In a separate spreadsheet there is a way 
shown how to properly include p-values in our statistical analysis. It then corresponds to what is obtained 
from statsmodels.

## Creating a summary table 

In [31]:
reg_summary = pd.DataFrame(data=["SAT","Rand 1,2,3"],columns=["Features"])
reg_summary = pd.DataFrame(data=x.columns.values, columns=["Features"]) # This is the same as the line above.
                                                            # This way is better for many features.
reg_summary

,Features
0,SAT
1,"Rand 1,2,3"


In [33]:
reg_summary["Coefficients"] = reg.coef_
reg_summary["p-values"] = p_values.round(3)

In [34]:
reg_summary

,Features,Coefficients,p-values
0,SAT,0.001654,0.000
1,"Rand 1,2,3",-0.008270,0.676


Note: p-values can tell us that a variable is redundant or needed. However, when two variables are needed, 
p-value cannot tell us how much one is more important than the other. 